In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv('/Users/xp/Downloads/crypto-markets.csv')
btc_data = df.loc[df['symbol'] == 'BTC']
sorted_btc_data = btc_data.sort_values('date', ascending= False)
sorted_btc_data

,slug,symbol,name,date,ranknow,open,high,low,close,volume,market,close_ratio,spread
2041,bitcoin,BTC,Bitcoin,2018-11-29,1,4269.00,4413.02,4145.77,4278.85,6.503348e+09,7.445102e+10,0.4980,267.25
2040,bitcoin,BTC,Bitcoin,2018-11-28,1,3822.47,4385.90,3822.47,4257.42,7.280280e+09,7.407256e+10,0.7720,563.43
2039,bitcoin,BTC,Bitcoin,2018-11-27,1,3765.95,3862.96,3661.01,3820.72,5.998720e+09,6.646897e+10,0.7908,201.95
2038,bitcoin,BTC,Bitcoin,2018-11-26,1,4015.07,4107.14,3643.92,3779.13,6.476900e+09,6.573929e+10,0.2919,463.22
2037,bitcoin,BTC,Bitcoin,2018-11-25,1,3880.78,4120.87,3585.06,4009.97,6.825640e+09,6.974927e+10,0.7930,535.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,bitcoin,BTC,Bitcoin,2013-05-02,1,116.38,125.60,92.28,105.21,0.000000e+00,1.168517e+09,0.3881,33.32
3,bitcoin,BTC,Bitcoin,2013-05-01,1,139.00,139.89,107.72,116.99,0.000000e+00,1.298955e+09,0.2882,32.17
2,bitcoin,BTC,Bitcoin,2013-04-30,1,144.00,146.93,134.05,139.00,0.000000e+00,1.542813e+09,0.3843,12.88
1,bitcoin,BTC,Bitcoin,2013-04-29,1,134.44,147.49,134.00,144.54,0.000000e+00,1.603769e+09,0.7813,13.49


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_error as mse
# Filter just for the BTC
btc_data = df.loc[df['symbol'] == 'BTC']

#Sorted the data according the date
sorted_btc_data = btc_data.sort_values('date', ascending=False)

#Get the previous days value
sorted_btc_data['T-1'] = sorted_btc_data['close'].shift(-1)
sorted_btc_data['T-2'] = sorted_btc_data['close'].shift(-2)
sorted_btc_data['T-3'] = sorted_btc_data['close'].shift(-3)
# Calculate the rolling closing price
sorted_btc_data['mean_price_3_days'] = sorted_btc_data['close'].rolling(window=3, min_periods=1).mean()
# Get the X and y value for Training and Testing
X = sorted_btc_data.drop('close', axis=1)
y = sorted_btc_data['close']
# Split the data for test and training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Get the required feature and response variable
X_train_new = X_train[['T-1', 'T-2', 'T-3']].fillna(0)
X_test_new = X_test[['T-1', 'T-2', 'T-3']].fillna(0)
# Normalize the data using the MinMaxScalar
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train_new)
X_test_normalized = scaler.transform(X_test_new)
#Define LinearRegression and fit the X and 
lm = LinearRegression()
lm.fit(X_train_normalized, y_train)
# Predict the target value with the test data
predicted = lm.predict(X_test_normalized)
# Calculate the metrics
mae = mean_absolute_error(y_test, predicted)
mse = mean_squared_error(y_test, predicted)
rmse = mean_squared_error(y_test, predicted, squared=False)
#Print the metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
#Print the mean closing price for the data
print(f'Mean of the close prices: {sorted_btc_data["close"].mean()}')


Mean Absolute Error (MAE): 95.48615373565481
Mean Squared Error (MSE): 62829.707908925695
Root Mean Squared Error (RMSE): 250.65854844574062
Mean of the close prices: 2287.287272282076


/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


The model's predictions are on average 95.49 USD away from the actual values (MAE), which is 4.2% of the average close price (2287.29 USD). This indicates a relatively good model